In [4]:
"""
This module provides RNN and CNN-RNN classifiers for video classification.
It includes:
  - VanilaRNN: A basic recurrent neural network for processing sequential data.
  - CNNRNNClassifier: A classifier that combines CNN-based feature extraction with a recurrent network.

Author: yumemonzo@gmail.com
Date: 2025-03-03
"""

import torch
import torch.nn as nn
from models import get_resnet18


class VanilaRNN(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, output_size: int, num_layers: int = 1) -> None:
        """
        Initialize a basic multi-layer RNN.

        Args:
            input_size (int): Dimension of the input feature vector.
            hidden_size (int): Dimension of the hidden state.
            output_size (int): Dimension of the output vector.
            num_layers (int): Number of stacked RNN layers. Defaults to 1.
        """
        super(VanilaRNN, self).__init__()
        self.hidden_size: int = hidden_size
        self.num_layers: int = num_layers
        
        # For each layer, create an input linear layer and a hidden state linear layer.
        # For the first layer, input dimension is input_size; for subsequent layers, it's hidden_size.
        self.input_layers = nn.ModuleList()
        self.hidden_layers = nn.ModuleList()
        
        for layer in range(num_layers):
            in_dim = input_size if layer == 0 else hidden_size
            self.input_layers.append(nn.Linear(in_dim, hidden_size))
            self.hidden_layers.append(nn.Linear(hidden_size, hidden_size))
        
        self.output_layer: nn.Linear = nn.Linear(hidden_size, output_size)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Process an input sequence through the multi-layer RNN.

        Args:
            x (torch.Tensor): Input tensor with shape (batch_size, seq_len, input_size).

        Returns:
            torch.Tensor: Output tensor with shape (batch_size, output_size).
        """
        batch_size, seq_len, _ = x.shape
        # Initialize hidden states for all layers.
        h = [torch.zeros(batch_size, self.hidden_size, device=x.device) for _ in range(self.num_layers)]
        
        # Process each time step.
        for t in range(seq_len):
            input_t = x[:, t, :]
            for layer in range(self.num_layers):
                # For the first layer, use the current input; for subsequent layers, use the previous layer's output.
                layer_input = input_t if layer == 0 else h[layer - 1]
                h[layer] = torch.tanh(self.input_layers[layer](layer_input) + self.hidden_layers[layer](h[layer]))
        
        # Use the last layer's hidden state for output.
        y = self.output_layer(h[-1])
        return y
    

class CNNRNNClassifier(nn.Module):
    def __init__(self, hidden_size: int, num_classes: int, num_layers: int = 1) -> None:
        """
        Initialize the CNN-RNN classifier.

        This classifier extracts features from video frames using a pretrained ResNet18 and
        processes the sequence of features using a multi-layer RNN.

        Args:
            hidden_size (int): Dimension of the RNN hidden state.
            num_classes (int): Number of output classes.
            num_layers (int): Number of stacked RNN layers. Defaults to 1.
        """
        super(CNNRNNClassifier, self).__init__()
        resnet18_model = get_resnet18()
        # Remove the final fully connected layer of ResNet18.
        self.feature_extractor: nn.Sequential = nn.Sequential(*list(resnet18_model.children())[:-1])
        # Input to RNN is a flattened feature map of size 512.
        self.rnn: VanilaRNN = VanilaRNN(512, hidden_size, num_classes, num_layers)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of the CNN-RNN classifier.

        Args:
            x (torch.Tensor): Input tensor with shape (batch_size, seq_len, C, H, W).

        Returns:
            torch.Tensor: Output logits with shape (batch_size, num_classes).
        """
        batch_size, seq_len, C, H, W = x.shape
        features = []
        for t in range(seq_len):
            # x[:, t] has shape (batch_size, C, H, W)
            x_t = self.feature_extractor(x[:, t])
            # Flatten the feature map.
            x_t = x_t.view(batch_size, -1)
            features.append(x_t)
        # Stack along the time dimension -> shape: (batch_size, seq_len, feature_size)
        features = torch.stack(features, dim=1)
        print(features.shape)

        return self.rnn(features)

In [5]:
input = torch.zeros((4, 50, 3, 224, 224))
label = torch.zeros((4,))

In [6]:
print(input.shape)
print(label.shape)

torch.Size([4, 50, 3, 224, 224])
torch.Size([4])


In [7]:
classifier = CNNRNNClassifier(256, 101, 4)

In [8]:
output = classifier(input)

torch.Size([4, 50, 512])
